<a href="https://colab.research.google.com/github/YutzMame/lecture-ai-engineering/blob/master/%E7%AC%AC3%E5%9B%9E%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察する。これにより、特別な学習なしでのモデルの限界を確認する。

2. **作成したテキストの活用**  
   リサーチを行い作成したテキストをモデルに学習させ、RAGを導入する。


## 扱う質問

「NintetndoSwich2」に関する質問を取り扱う

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用する。このモデルは、リリース時期の関係上、「Switch2」が発表される前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [2]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [3]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 2)
Receiving objects: 100% (81/81), 1.34 MiB | 2.45 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [4]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import random
random.seed(0)

In [7]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [8]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [9]:
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    response = generate_output(question)
    print(f"質問: {question}")
    print(f"回答: {response}\n")

質問: Nintendo Switch2の国内専用本体価格はいくらですか
回答:  残念ながら、Nintendo Switch 2の国内専用本体価格はまだ発表されていません。


 




質問: Nintendo Switchの本体価格はいくらですか
回答:  Nintendo Switch本体の価格は、**約19,800円～21,800円**です。 

 
 
 
 




質問: Nintendo Switch2の発売日はいつですか
回答: 残念ながら、Nintendo Switch 2の発売日は現在発表されていません。 


 




質問: Nintendo SwitchとNintendo Switch2の違いを教えてください
回答: ## Nintendo SwitchとNintendo Switch2の違い

残念ながら、Nintendo Switch2の具体的な情報はまだ公開されていません。 

しかし、いくつかの噂や推測に基づいて、Switch2がどのような進化をするのかを想像してみましょう。

**Switch2の期待される特徴:**

* **より高性能なハードウェア:**  Switch2は、Switchの性能を大幅に向上させることが期待されます。CPUやGPUの性能向上、より高速な通信、より高画質のゲームプレイを実現する可能性があります。
* **VR対応:**  VR技術の進化により、Switch2ではVR対応が期待されます。
* **より高画質のゲーム:**  Switch2は、より高画質のゲームをプレイできる可能性があります。
* **より進化した操作性:**  Switch2では、より革新的な操作性を実現する可能性があります。例えば、より自然な手動操作や、より複雑な操作に対応する技術が導入されるかもしれません。
* **新しいゲーム体験:**  Switch2では、従来のゲーム体験に加え、新しいゲーム体験を提供する可能性があります。例えば、よりリアルな世界観や、より没入感のあるゲーム体験を提供する技術が導入されるかもしれません。
* **

質問: Nintendo Switchの発売日はいつですか
回答:  Nintendo Switchの発売日は **2017年3月3日** です。 😊







## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Nintendo Switch2」についての情報を知らない旨の回答をし、ハルシネーションは起こさなかった。モデルの作成時期の関係で、Switch2の情報を持たないと考えられる。

---

# 2. 作成したテキストの活用

## 2.1 作成したテキストをソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入する：

* **知識ソース**: 自身でリサーチして作成したテキスト
* **目的**: モデルに「NintendoSwitch2」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**:作成したテキストを使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [10]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [11]:
with open("/content/lecture-ai-engineering/day3/data/about_switch2.txt", "r") as f:
  raw_writedown = f.read()

In [12]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[40])

ドキュメントサイズ:  80
ドキュメントの例: 
 発売されるのは『ゼルダの伝説　ブレス オブ ザ ワイルド』、『ゼルダの伝説　ティアーズ オブ ザ キングダム』、『メトロイドプライム4 ビヨンド』、『星のカービィ ディスカバリー』、『ポケモンレジェンズZ-A』など


In [13]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[78.79585266113281, 77.43077087402344, 67.54423522949219, 71.13761901855469, 74.60729217529297, 68.56951141357422, 66.97605895996094, 67.78649139404297, 84.27302551269531, 69.49694061279297, 71.57652282714844, 67.57599639892578, 82.29217529296875, 76.49646759033203, 76.24617767333984, 76.72381591796875, 71.3587646484375, 75.36528778076172, 74.86466217041016, 66.39753723144531, 67.20764923095703, 66.77053833007812, 67.10160827636719, 66.38654327392578, 65.05815887451172, 67.47876739501953, 76.964599609375, 68.81197357177734, 76.7423324584961, 68.98033905029297, 76.5891342163086, 69.84087371826172, 73.93375396728516, 71.57652282714844, 67.57599639892578, 69.35090637207031, 66.83055877685547, 72.60383605957031, 67.0440444946289, 83.22439575195312, 78.46623992919922, 79.72152709960938, 76.00529479980469, 78.34591674804688, 78.34591674804688, 78.78157043457031, 69.39987182617188, 80.57124328613281, 74.74620056152344, 74.9588394165039, 80.2362060546875, 69.46419525146484, 70.63797760009766,

In [14]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 84.27302551269531)
Switch2の発売日は2025年6月5日
　Switch2に内蔵されているのは、リチウムイオンバッテリー（電池容量 5220mAh） 


取得したドキュメント2: (Score: 83.22439575195312)
Nintendo Switchのソフトを高性能なNintendo Switch 2で遊べる、“Nintendo Switch 2エディション”が複数のタイトルで展開される 


取得したドキュメント3: (Score: 82.29747772216797)
本体と同日にNintendo Switch 2カメラが発売 


取得したドキュメント4: (Score: 82.29217529296875)
switch２は、日本語・国内専用本体は49980円［税込］ 


取得したドキュメント5: (Score: 80.57124328613281)
Nintendo Switch 2では、初代Nintendo Switch向けタイトルもプレイ可能 




In [15]:
questions = [
    "Nintendo Switch2の国内専用本体価格はいくらですか",
    "Nintendo Switchの本体価格はいくらですか",
    "Nintendo Switch2の発売日はいつですか",
    "Nintendo SwitchとNintendo Switch2の違いを教えてください",
    "Nintendo Switchの発売日はいつですか"
]

for question in questions:
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)
    references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
    query =  f"[参考資料]\n{references}\n\n[質問]"+question
    response = generate_output(query)
    print(f"質問: {question}")
    print(f"回答: {response}\n")

質問: Nintendo Switch2の国内専用本体価格はいくらですか
回答: Nintendo Switch2の国内専用本体価格は **49980円（税込）** です。  




質問: Nintendo Switchの本体価格はいくらですか
回答: Nintendo Switchの本体価格は **49980円（税込）** です。  




質問: Nintendo Switch2の発売日はいつですか
回答: Nintendo Switch2の発売日は2025年6月5日です。  




質問: Nintendo SwitchとNintendo Switch2の違いを教えてください
回答: Nintendo Switch2とNintendo Switchの違いは以下の通りです。

**1.性能向上:**
   * **CPU/GPU:**  より高速で高性能なCPU/GPUを搭載
   * **リチウムイオンバッテリー:**  電池容量が5220mAhとなり、より長時間使用が可能
   * **画面:**  画面の解像度や刷新率が向上

**2.ソフト対応:**
   * **Nintendo Switch 2エディション:**  複数のタイトルで、Nintendo Switch 2専用の機能や画質でプレイできる
   * **初代Nintendo Switch向けタイトル:**  Nintendo Switch2でもプレイ可能

**3.発売日:**
   * **Nintendo Switch2:**  2025年6月5日


 




質問: Nintendo Switchの発売日はいつですか
回答: Nintendo Switchの発売日は、**2017年3月3日**です。  







### 結果 (初期RAG実装)

RAG導入後はモデル作成時以降のトピックについての説明を、参考資料をもとにして正確にすることができていた。しかし、RAG実装前に回答出来ていた質問「Nintendo Switchの本体価格はいくらですか」に対して、「Nintendo Switchの本体価格は **49980円（税込）** です。」という回答が見られた。これは、 Switch2の本体価格であり、モデルが元々持っていた知識を参照せず、参考資料にある知識を参照した、さらに、Switch2と Switchという似た単語の区別をモデルができなかったことで起きたと考えられる。解決法としては、モデルにSwitch2と Switchが別のものであることを念入りに学習させることなどが挙げられる。